In [1]:
from sentence_recognition import *

Using TensorFlow backend.


In [2]:
filename = '../data/sentiment_labelled_sentences/amazon_cells_labelled.txt'
lines = load_data(filename)

In [3]:
lines = lines.split('\n')
lines = [line[:-2] for line in lines]
lines = lines[0:-1]

In [4]:
tokenized_lines = tokenize_words_in_sentences(lines)

In [16]:
false_tokenized_lines = []
for _ in range(len(tokenized_lines)):
    false_tokenized_lines.append(corrupt_sentence_generator(tokenized_lines))

ML inputs

In [17]:
X = false_tokenized_lines + tokenized_lines
y = [0 for _ in range(len(false_tokenized_lines))] + \
    [1 for _ in range(len(tokenized_lines))]

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.1,
                                                    random_state=42)

Saving datasets

In [19]:
save_dataset([X_train,y_train], 'train_s0.pkl')
save_dataset([X_test,y_test], 'test_s0.pkl')

Saved: train_s0.pkl
Saved: test_s0.pkl


Load saved datasets

In [20]:
trainLines, trainLabels = load_dataset('train_s0.pkl')

In [21]:
tokenizer = create_tokenizer(trainLines)

In [36]:
# calculate max document length
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
print(trainX.shape)

Max document length: 29
Vocabulary size: 1725
(1800, 29)


In [37]:
# define model
model = define_model(length, vocab_size)
# fit model
model.fit(trainX, np.array(trainLabels), epochs=10, batch_size=16)
# save the model
model.save('model_single_channel.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 29)                0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 29, 100)           172500    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 26, 32)            12832     
_________________________________________________________________
dropout_5 (Dropout)          (None, 26, 32)            0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 13, 32)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 416)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                4170      
__________

Testing accuracy

In [39]:
from keras.models import load_model


# load datasets
trainLines, trainLabels = load_dataset('train_s0.pkl')
testLines, testLabels = load_dataset('test_s0.pkl')

# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
testX = encode_text(tokenizer, testLines, length)
print(trainX.shape, testX.shape)

# load the model
model = load_model('model_single_channel.h5')

# evaluate model on training dataset
loss, acc = model.evaluate(trainX, np.array(trainLabels), verbose=0)
print('Train Accuracy: %f' % (acc*100))

# evaluate model on test dataset dataset
loss, acc = model.evaluate(testX, np.array(testLabels), verbose=0)
print('Test Accuracy: %f' % (acc*100))

Max document length: 29
Vocabulary size: 1725
(1800, 29) (200, 29)
Train Accuracy: 99.444444
Test Accuracy: 66.500000


In [54]:
def predictor(model, sentence, tokenized=False):
    if not tokenized:
        sentence_tmp = [tokenize_sentence(sentence)]
    else:
        sentence_tmp = sentence
    score = model.predict(encode_text(tokenizer, sentence_tmp, length))[0]
    if score > 0.5:
        print('Sentence!')
    else:
        print('Not a sentence!')

In [55]:
predictor(model, 'This is a sentence.')

Sentence!


In [76]:
predictor(model, 'This a sentence.')

Not a sentence!


In [62]:
sentence_tmp = corrupt_sentence_generator(tokenized_lines)
print(' '.join(word for word in sentence_tmp))
predictor(model, sentence_tmp, True)

Pros anything Good but
Not a sentence!


In [78]:
tokenize_sentence

TypeError: 'function' object is not subscriptable